Notebook to transform the Gridded Livestock of the World (GLW) cattle density TIFFs data into a workable DataFrame with country, administrative boundary1, administrative boundary2 and number of heads(animals). 
Validation of the 2020 density data (totals per country) with the FAO STATS for same country/specie/year.
% Calculation of each administrative level against total population to be used as future figure to calculate future density assuming the density parameter does not vary much


1. Necessary Libraries:

In [2]:
#pip install rasterio geopandas rasterstats
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
from rasterio.plot import show
from rasterstats import zonal_stats

2. Load Administrative Boundaries:
Download: Obtain a shapefile or GeoJSON containing administrative boundaries with country and region information (second administrative level). You can find suitable data from sources like GADM (https://gadm.org/). Load into GeoPandas:

In [3]:
admin_boundaries = gpd.read_file("gadm_410.gpkg")

In [3]:
# Display column names to inspect variable names
print(admin_boundaries.columns)

Index(['UID', 'GID_0', 'NAME_0', 'VARNAME_0', 'GID_1', 'NAME_1', 'VARNAME_1',
       'NL_NAME_1', 'ISO_1', 'HASC_1', 'CC_1', 'TYPE_1', 'ENGTYPE_1',
       'VALIDFR_1', 'GID_2', 'NAME_2', 'VARNAME_2', 'NL_NAME_2', 'HASC_2',
       'CC_2', 'TYPE_2', 'ENGTYPE_2', 'VALIDFR_2', 'GID_3', 'NAME_3',
       'VARNAME_3', 'NL_NAME_3', 'HASC_3', 'CC_3', 'TYPE_3', 'ENGTYPE_3',
       'VALIDFR_3', 'GID_4', 'NAME_4', 'VARNAME_4', 'CC_4', 'TYPE_4',
       'ENGTYPE_4', 'VALIDFR_4', 'GID_5', 'NAME_5', 'CC_5', 'TYPE_5',
       'ENGTYPE_5', 'GOVERNEDBY', 'SOVEREIGN', 'DISPUTEDBY', 'REGION',
       'VARREGION', 'COUNTRY', 'CONTINENT', 'SUBCONT', 'geometry'],
      dtype='object')


In [4]:
# Ensure correct CRS
admin_boundaries = admin_boundaries.to_crs('EPSG:4326')

3. Load Animal Density Rasters:

In [4]:
cattle_density_raster = rasterio.open('GLW4-2020.D-DA.CTL.tif')
#buffalo_density_raster = rasterio.open('GLW4-2020.D-DA.BFL.tif')
#goat_density_raster = rasterio.open('GLW4-2020.D-DA.GTS.tif')
#sheep_density_raster= rasterio.opne('GLW4-2020.D-DA.SHP.tif')
#pig_density_raster= rasterio.opne('GLW4-2020.D-DA.PGS.tif')


4. Perform Zonal Statistics

In [ ]:
# Define chunk size (number of features in admin_boundaries per chunk)
chunk_size = 1000

# Get the total number of features
total_features = len(admin_boundaries)

# Initialize an empty list to store results
all_stats = []

# Loop through chunks
for i in range(0, total_features, chunk_size):
    # Select chunk of admin_boundaries
    admin_chunk = admin_boundaries.iloc[i:i + chunk_size]
    
    # Perform zonal statistics on the chunk
    stats_chunk = zonal_stats(admin_chunk, cattle_density_raster, stats="sum", geojson_out=True)
    
    # Append results to the list
    all_stats.extend(stats_chunk)

# Now all_stats contains zonal statistics for all chunks

In [ ]:
# Perform zonal statistics chunked above as notebook crashes
#stats = zonal_stats(admin_boundaries, cattle_density_raster, stats="sum", geojson_out=True)


5. Convert to DataFrame and Export to CSV

In [ ]:


# Convert to DataFrame
results_df = pd.DataFrame(stats)

# Extract relevant attributes from GeoDataFrame (if needed)
results_df['NAME_1'] = admin_boundaries['NAME_1']
results_df['NAME_2'] = admin_boundaries['NAME_2']




6. Load FAO Animal Stats and validate 

In [ ]:
# Specify the path to your CSV file
csv_file = "FAO_animal_stats.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file)

# Display the columns to understand the structure of the data
print(df.columns)

# Filter for the year 2020 and specific species
species_of_interest = ['Cattle', 'Buffalo', 'Pigs', 'Goats', 'Sheep']

fao_totals = df[(df['Year'] == 2020) & (df['Item'].isin(species_of_interest))]

# Display the first few rows to verify the data
print(filtered_data.head())


In [ ]:
# Assuming results_df is the DataFrame from zonal statistics
# Group by NAME_1 (first administrative level) and sum the animal counts for each species
zonal_totals = results_df.groupby('NAME_1')[['Cattle', 'Buffalo', 'Pigs', 'Goats', 'Sheep']].sum()


In [ ]:
# Merge zonal_totals and fao_totals on the index (assuming NAME_1 and Area match)
comparison = pd.merge(zonal_totals, fao_totals, left_index=True, right_index=True, suffixes=('_zonal', '_fao'))

# Print the comparison
print(comparison)


In [ ]:
# Function to compare and summarize
def compare_summary(zonal_totals, fao_totals):
    # Merge datasets
    comparison = pd.merge(zonal_totals, fao_totals, left_index=True, right_index=True, suffixes=('_zonal', '_fao'))
    
    # Initialize counters
    same_count = 0
    different_count = 0
    
    # Loop through each species
    for species in zonal_totals.columns:
        # Compare values
        is_same = comparison[species + '_zonal'] == comparison['Value']
        
        # Count same and different
        same_count += is_same.sum()
        different_count += (~is_same).sum()
    
    return same_count, different_count

# Call the function to get summary
same_count, different_count = compare_summary(zonal_totals, fao_totals)

# Print summary
print(f"Number of entries that are the same: {same_count}")

7.Create Percentage of each region against total population

In [ ]:
# Merge zonal_totals with fao_totals
merged_data = pd.merge(zonal_totals, fao_totals, left_on='NAME_1', right_on='Country', suffixes=('_zonal', '_fao'))


In [ ]:
# Calculate percentage for each species
species_of_interest = ['Cattle', 'Buffalo', 'Pigs', 'Goats', 'Sheep']

for species in species_of_interest:
    # Calculate percentage for level 1 (NAME_1)
    merged_data[f'{species}_percent_level1'] = (merged_data[species] / merged_data[f'{species}_fao']) * 100
    
    # Calculate percentage for level 2 (NAME_2 or district)
    merged_data[f'{species}_percent_level2'] = (merged_data[species] / merged_data[f'{species}_fao']) * 100

# Select columns of interest
result_columns = ['Country', 'NAME_1', 'NAME_2'] + [f'{species}_percent_level1' for species in species_of_interest] + [f'{species}_percent_level2' for species in species_of_interest]
result_df = merged_data[result_columns]

# Display or export result_df as needed
print(result_df)


In [ ]:
# Export to CSV
results_df.to_csv("zonal_statistics.csv", index=False)